In [4]:
from google.colab import drive
                                                                                                                                                                                                                                                                    drive.mount('/content/drive')
fh= open('/content/drive/MyDrive/Colab Notebooks/POS TAGGING/train_data/train_data.txt','r')

Mounted at /content/drive


In [87]:
#Importing libraries
import random
import nltk, re, pprint
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import pprint, time
import random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize

In [88]:
fh= open('train_data/train_data.txt','r')

In [121]:
fh_test= open('test_data/test_data.txt','r')

In [89]:
train_data=fh.read().splitlines()

In [122]:
testset_data=fh_test.read().splitlines()

In [90]:
len(train_data)

14963

In [91]:
train_data[:5]

['यह/DEM एशिया/NNP की/PSP सबसे/INTF बड़ी/JJ मस्जिदों/NN में/PSP से/PSP एक/QC है/VM ।/SYM',
 'इसे/PRP नवाब/NN शाहजेहन/NNP ने/PSP बनवाया/VM था/VAUX ।/SYM',
 'इसका/PRP प्रवेश/NN द्वार/NN दो/QC मंजिला/JJ है/VM ।/SYM',
 'जिसमें/PRP चार/QC मेहराबें/NN हैं/VM और/CC मुख्य/JJ प्रार्थना/NN हॉल/NN में/PSP जाने/VM के/PSP लिए/PSP 9/QC प्रवेश/NN द्वार/NN हैं/VM ।/SYM',
 'पूरी/JJ इमारत/NN बेहद/INTF खूबसूरत/JJ है/VM ।/SYM']

In [92]:
new_data=[]

In [93]:
for i in range(len(train_data)):
    new_li=train_data[i].split(" ");
    li=[]
    for j in new_li:
        if j == '।/SYM':
          tup=('.','SYM')
          li.append(tup)
        else:
          li.append(tuple(j.split('/')))
    new_data.append(li)

In [127]:
newtestset_data=[]

In [128]:
for i in range(len(testset_data)):
    new_li=testset_data[i].split(" ");
    li=[]
    for j in new_li:
        if j == '।/SYM':
          tup=('.','SYM')
          li.append(tup)
        else:
          li.append(tuple(j.split('/')))
    newtestset_data.append(li)

In [94]:
len(new_data)

14963

In [129]:
len(newtestset_data)

1684

In [95]:
# Splitting into train and test
random.seed(1234)
train_set, test_set = train_test_split(new_data,test_size=0.05)

print(len(train_set))
print(len(test_set))


14214
749


In [96]:
# Getting list of tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
len(train_tagged_words)


300284

In [97]:
# Sequence of all words stored in tokens list
tokens = [pair[0] for pair in train_tagged_words]
tokens[:1]

['इस', 'लड़की', 'के', 'स्वागत', 'में', 'घर', 'की', 'तस्वीर', 'ही', 'बदल']

In [98]:
# vocabulary: Unique tokens
V = set(tokens)
print(len(V))

17719


In [99]:
# number of tags
T = set([pair[1] for pair in train_tagged_words])
len(T)

26

# Emmision probabilities

In [100]:
# computing P(w/t) and storing in T x V matrix
t = len(T)
v = len(V)
w_given_t = np.zeros((t, v))

In [101]:
# compute word given tag: Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    # saving 
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag, count_tag)

# Transition Probabilities

In [102]:
# compute tag given tag: tag2(t2) given tag1 (t1), i.e. Transition Probability

def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [103]:
T

{'',
 'CC',
 'DEM',
 'INJ',
 'INTF',
 'JJ',
 'NEG',
 'NN',
 'NNP',
 'NST',
 'PRP',
 'PRPC',
 'PSP',
 'QC',
 'QF',
 'QFC',
 'QO',
 'RB',
 'RBC',
 'RDP',
 'RP',
 'SYM',
 'UNK',
 'VAUX',
 'VM',
 'WQ'}

In [104]:
# examples
print(t2_given_t1(t2='NN', t1='JJ'))
print(t2_given_t1('।', 'RBC'))



(9325, 17097)
(0, 20)


In [105]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [106]:
tags_matrix

array([[6.45225821e-03, 0.00000000e+00, 1.84264496e-01, 1.00035009e-04,
        1.38548492e-02, 1.75061275e-03, 3.80133046e-03, 8.05281848e-03,
        6.26719370e-02, 6.58230409e-02, 5.80203068e-03, 2.06572302e-02,
        2.31581051e-02, 3.20162058e-01, 7.65267853e-03, 5.00175047e-05,
        1.00035009e-04, 5.00175047e-05, 5.59695885e-02, 2.50087527e-04,
        2.06622317e-01, 3.00105050e-04, 5.00175047e-05, 2.90101534e-03,
        0.00000000e+00, 9.45330877e-03],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 6.66666687e-01, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        3.33333343e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [4.28906688e-03, 0.00000000e+00, 7.74459839e-02

In [107]:
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index=list(T))

In [108]:
tags_df

,SYM,,PRP,INJ,RB,WQ,QO,VM,JJ,CC,...,UNK,NEG,DEM,PRPC,NN,INTF,VAUX,RP,RBC,RDP
SYM,0.006452,0.000000,0.184264,0.000100,0.013855,0.001751,0.003801,0.008053,0.062672,0.065823,...,0.000100,0.000050,0.055970,0.000250,0.206622,0.000300,0.000050,0.002901,0.000000,0.009453
,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000
PRP,0.004289,0.000000,0.077446,0.000000,0.007283,0.002266,0.004289,0.121227,0.097273,0.002751,...,0.000728,0.006717,0.025815,0.000809,0.322085,0.003480,0.000000,0.040139,0.000000,0.000081
INJ,0.000000,0.000000,0.666667,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
RB,0.060109,0.000000,0.068696,0.000000,0.001561,0.003123,0.007026,0.147541,0.103825,0.001561,...,0.000781,0.010929,0.031226,0.000000,0.266198,0.000000,0.000000,0.128806,0.000000,0.005464
WQ,0.010453,0.000000,0.083624,0.000000,0.003484,0.000000,0.000000,0.376307,0.052265,0.003484,...,0.000000,0.045296,0.024390,0.000000,0.278746,0.000000,0.003484,0.006969,0.000000,0.000000
QO,0.000000,0.000000,0.004141,0.000000,0.000000,0.000000,0.000000,0.002070,0.051760,0.024845,...,0.000000,0.000000,0.008282,0.000000,0.780538,0.004141,0.000000,0.006211,0.000000,0.000000
VM,0.182918,0.000000,0.016042,0.000000,0.000721,0.000157,0.000313,0.024596,0.010496,0.141089,...,0.000094,0.001974,0.002444,0.000125,0.033651,0.000345,0.422829,0.005608,0.000031,0.000031
JJ,0.013160,0.000000,0.002457,0.000000,0.000058,0.000292,0.000175,0.295198,0.039832,0.012809,...,0.000000,0.021056,0.002983,0.000058,0.545417,0.000175,0.000000,0.002808,0.000000,0.000058
CC,0.017021,0.000000,0.187500,0.000000,0.008333,0.005142,0.003812,0.006826,0.102926,0.012057,...,0.000000,0.000532,0.045656,0.000443,0.271011,0.000975,0.000000,0.005940,0.000000,0.000000


# vanilla viterbi

In [112]:
# Viterbi Heuristic
def Viterbi_vanilla(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
#                 print(tags_df.loc['SYM', tag])
                transition_p = tags_df.loc['SYM', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        #print('word is {} and list is {}'.format(word,p))
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))


# Viterbi smoothing

In [113]:
def Viterbi_smoothed(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['SYM', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = 0.000001+word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]+0.000001*len(T)
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        #print('word is {} and list is {}'.format(word,p))
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

# Viterbi using transition only in case of emission is zero 
## i.e if a test word never appear in training data.

In [115]:
# Viterbi if emission is zero
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['SYM', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            if word in tokens: 
                emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            else:
                emission_p=1
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        #print('word is {} and list is {}'.format(word,p))
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

# viterbi using mostly used tag

In [114]:
def Viterbi_common(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['SYM', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            if word in tokens: 
                emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            else:
                emission_p=1
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        #print('word is {} and list is {}'.format(word,p))
        pmax = max(p)
        if pmax==0.0:
            state_word='NNP'
            state.append(state_word)
            #print(word,':',state_word)
        else:
        # getting state for which probability is maximum
            state_max = T[p.index(pmax)] 
            state.append(state_max)
    return list(zip(words, state))


# Preparing test data

In [116]:

test_run_base = [tup for sent in test_set for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_set for tup in sent]


In [130]:
# final_test_run_base = [tup for sent in newtestset_data for tup in sent]

# # list of untagged words
# final_test_tagged_words = [tup[0] for sent in newtestset_data for tup in sent]

## 1. TEST on vanilla viterbi

In [65]:
# tagging the test sentences
start = time.time()
tagged_seq_vanilla = Viterbi_vanilla(test_tagged_words)
check = [i for i, j in zip(tagged_seq_vanilla, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq_vanilla)
end = time.time()
print('Time Taken:',end-start)
print('Accuracy is :',accuracy)

0.004272674
0.0
0.1916877
0.015148573
0.004661099
0.0033016119
0.010681686
0.06758594
0.06525539
0.0077684987
0.020780735
0.0155369975
0.289765
0.008933773
0.0
0.0
0.0
0.06622645
0.0005826374
0.21130316
0.00019421247
0.0
0.0033016119
0.0
0.012818023
Time Taken: 794.8016953468323
Accuracy is : 0.887414079165679


In [66]:
print('Vanilla Viterbi-Incorrect tagged cases are:')
incorrect_tagged_cases = [[test_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq_vanilla, test_run_base)) if j[0]!=j[1]]
[i[1]for i in incorrect_tagged_cases]

Vanilla Viterbi-Incorrect tagged cases are:


[(('तो', 'PRP'), ('तो', 'CC')),
 (('उस', 'DEM'), ('उस', 'PRP')),
 (('पर', 'NN'), ('पर', 'PSP')),
 (('पौंड', 'SYM'), ('पौंड', 'NN')),
 (('लिखित', 'JJ'), ('लिखित', 'NN')),
 (('में', 'SYM'), ('में', 'PSP')),
 (('लेकर', 'VM'), ('लेकर', 'PSP')),
 (('बिता', 'SYM'), ('बिता', 'VM')),
 (('सकते', 'NN'), ('सकते', 'VAUX')),
 (('हैं', 'VM'), ('हैं', 'VAUX')),
 (('गहन', 'SYM'), ('गहन', 'JJ')),
 (('की', 'PSP'), ('की', 'VM')),
 (('1282', 'SYM'), ('1282', 'NNP')),
 (('वंशीय', 'SYM'), ('वंशीय', 'JJ')),
 (('राजा', 'NNP'), ('राजा', 'NN')),
 (('ईमेल', 'NNP'), ('ईमेल', 'NN')),
 (('वायुसेवा', 'SYM'), ('वायुसेवा', 'NN')),
 (('विदेश', 'NNP'), ('विदेश', 'NN')),
 (('मशविरे', 'SYM'), ('मशविरे', 'NN')),
 (('रजामंदी', 'SYM'), ('रजामंदी', 'NN')),
 (('उस', 'DEM'), ('उस', 'PRP')),
 (('पर', 'NN'), ('पर', 'PSP')),
 (('राष्ट्र', 'NNP'), ('राष्ट्र', 'NN')),
 (('२०वीं', 'SYM'), ('२०वीं', 'QO')),
 (('अखंडता', 'SYM'), ('अखंडता', 'NN')),
 (('पिरोने', 'SYM'), ('पिरोने', 'VM')),
 (('सिलसिलेवार', 'SYM'), ('सिलसिलेवार', 'RB')),
 

## 2.VITERBI WITH SMOOTHING

In [73]:
# tagging the test sentences
start = time.time()
tagged_seq_smooth = Viterbi_smoothed(test_tagged_words)

In [74]:
check = [i for i, j in zip(tagged_seq_smooth, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq_smooth)
end = time.time()
print('Time Taken:',end-start)
print('Accuracy is :',accuracy)

Time Taken: 832.6852796077728
Accuracy is : 0.8909694240341313


In [75]:
print('Laplace smoothed Viterbi-Incorrect tagged cases are:')
incorrect_tagged_cases = [[test_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq_smooth, test_run_base)) if j[0]!=j[1]]
[i[1]for i in incorrect_tagged_cases]

Laplace smoothed Viterbi-Incorrect tagged cases are:


[(('तो', 'PRP'), ('तो', 'CC')),
 (('उस', 'DEM'), ('उस', 'PRP')),
 (('पर', 'NN'), ('पर', 'PSP')),
 (('चित्र', 'VAUX'), ('चित्र', 'NN')),
 (('लिखित', 'JJ'), ('लिखित', 'NN')),
 (('में', 'NN'), ('में', 'PSP')),
 (('जानबूझकर', 'PSP'), ('जानबूझकर', 'RB')),
 (('लेकर', 'VM'), ('लेकर', 'PSP')),
 (('बिता', 'PSP'), ('बिता', 'VM')),
 (('सकते', 'NN'), ('सकते', 'VAUX')),
 (('हैं', 'VM'), ('हैं', 'VAUX')),
 (('गहन', 'NN'), ('गहन', 'JJ')),
 (('छानबीन', 'PSP'), ('छानबीन', 'NN')),
 (('की', 'PSP'), ('की', 'VM')),
 (('1282', 'PSP'), ('1282', 'NNP')),
 (('वंशीय', 'PSP'), ('वंशीय', 'JJ')),
 (('हमीर', 'PSP'), ('हमीर', 'NNP')),
 (('ईमेल', 'NNP'), ('ईमेल', 'NN')),
 (('विदेश', 'NNP'), ('विदेश', 'NN')),
 (('मशविरे', 'NNP'), ('मशविरे', 'NN')),
 (('उस', 'DEM'), ('उस', 'PRP')),
 (('पर', 'NN'), ('पर', 'PSP')),
 (('मलेशिया', 'SYM'), ('मलेशिया', 'NNP')),
 (('राष्ट्र', 'NNP'), ('राष्ट्र', 'NN')),
 (('२०वीं', 'NN'), ('२०वीं', 'QO')),
 (('पुण्यतिथि', 'PSP'), ('पुण्यतिथि', 'NN')),
 (('अखंडता', 'NNP'), ('अखंडता', 'NN')),
 

## 3.Testing model on viterbi with unknown word given mostly used tag

In [117]:
start = time.time()
tagged_seq_co = Viterbi_common(test_tagged_words)
check = [i for i, j in zip(tagged_seq_co, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq_co)
end = time.time()
print('Time Taken:',end-start)
print('Accuracy is :',accuracy)

Time Taken: 13914.598916769028
Accuracy is : 0.9285178236397749


In [118]:
print('Laplace smoothed Viterbi-Incorrect tagged cases are:')
incorrect_tagged_cases = [[test_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq_co, test_run_base)) if j[0]!=j[1]]
[i[1]for i in incorrect_tagged_cases]

Laplace smoothed Viterbi-Incorrect tagged cases are:


[(('वामदल', 'NN'), ('वामदल', 'NNP')),
 (('फ्रंट', 'NNP'), ('फ्रंट', 'NN')),
 (('पहले', 'NST'), ('पहले', 'QO')),
 (('कुछ', 'PRP'), ('कुछ', 'QF')),
 (('समन्वय', 'NN'), ('समन्वय', 'JJ')),
 (('की', 'PSP'), ('की', 'VM')),
 (('गई', 'VM'), ('गई', 'VAUX')),
 (('संबंधी', 'JJ'), ('संबंधी', 'PSP')),
 (('मुख्य', 'JJ'), ('मुख्य', 'NN')),
 (('बरकत', 'VAUX'), ('बरकत', 'NNP')),
 (('की', 'PSP'), ('की', 'VM')),
 (('थी', 'VM'), ('थी', 'VAUX')),
 (('अतिरिक्त', 'JJ'), ('अतिरिक्त', 'NN')),
 (('भौंडसी', 'PSP'), ('भौंडसी', 'NNP')),
 (('चौर', 'NN'), ('चौर', 'NNP')),
 (('सरोवर', 'NN'), ('सरोवर', 'NNP')),
 (('अलग', 'RDP'), ('अलग', 'JJ')),
 (('रूप', 'NN'), ('रूप', 'PSP')),
 (('शूट', 'PSP'), ('शूट', 'UNK')),
 (('मंत्रालय', 'NNP'), ('मंत्रालय', 'NN')),
 (('राष्ट्रीय', 'JJ'), ('राष्ट्रीय', 'NNP')),
 (('उपभोक्ता', 'NN'), ('उपभोक्ता', 'NNP')),
 (('विवाद', 'NN'), ('विवाद', 'NNP')),
 (('निवारण', 'PSP'), ('निवारण', 'NNP')),
 (('आयोग', 'NN'), ('आयोग', 'NNP')),
 (('आईएनजी', 'NN'), ('आईएनजी', 'NNP')),
 (('वैश्य', 'PSP'), ('

## 4. Testing model on Viterbi choosing only transisition probability in case emission probability is zero

In [78]:
start = time.time()
tagged_seq_t = Viterbi(test_tagged_words)
check = [i for i, j in zip(tagged_seq_t, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq_t)
end = time.time()
print('Time Taken:',end-start)
print('Accuracy is :',accuracy)
print('Laplace smoothed Viterbi-Incorrect tagged cases are:')
incorrect_tagged_cases = [[test_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq_t, test_run_base)) if j[0]!=j[1]]
[i[1]for i in incorrect_tagged_cases]

Time Taken: 774.4879562854767
Accuracy is : 0.9068499644465513
Laplace smoothed Viterbi-Incorrect tagged cases are:


[(('तो', 'PRP'), ('तो', 'CC')),
 (('उस', 'DEM'), ('उस', 'PRP')),
 (('पर', 'NN'), ('पर', 'PSP')),
 (('लिखित', 'JJ'), ('लिखित', 'NN')),
 (('में', 'SYM'), ('में', 'PSP')),
 (('लेकर', 'VM'), ('लेकर', 'PSP')),
 (('बिता', 'PSP'), ('बिता', 'VM')),
 (('सकते', 'NN'), ('सकते', 'VAUX')),
 (('हैं', 'VM'), ('हैं', 'VAUX')),
 (('गहन', 'NN'), ('गहन', 'JJ')),
 (('की', 'PSP'), ('की', 'VM')),
 (('1282', 'PSP'), ('1282', 'NNP')),
 (('वंशीय', 'PSP'), ('वंशीय', 'JJ')),
 (('ईमेल', 'NNP'), ('ईमेल', 'NN')),
 (('विदेश', 'NNP'), ('विदेश', 'NN')),
 (('मशविरे', 'NNP'), ('मशविरे', 'NN')),
 (('उस', 'DEM'), ('उस', 'PRP')),
 (('पर', 'NN'), ('पर', 'PSP')),
 (('राष्ट्र', 'NNP'), ('राष्ट्र', 'NN')),
 (('२०वीं', 'NN'), ('२०वीं', 'QO')),
 (('अखंडता', 'NNP'), ('अखंडता', 'NN')),
 (('पिरोने', 'NN'), ('पिरोने', 'VM')),
 (('सिलसिलेवार', 'VAUX'), ('सिलसिलेवार', 'RB')),
 (('२६', 'NNP'), ('२६', 'QC')),
 (('लगतीं', 'VM'), ('लगतीं', 'VAUX')),
 (('प्रकाश', 'NN'), ('प्रकाश', 'NNP')),
 (('अपहरण', 'NN'), ('अपहरण', 'NNP')),
 (('इस', 'DE

# Final testing on test data set


In [142]:
# start = time.time()
# final_tagged_seq_co = Viterbi_common(final_test_tagged_words)

In [143]:
# check = [i for i, j in zip(final_tagged_seq_co, test_run_base) if i == j] 
# accuracy = len(check)/len(final_tagged_seq_co)
# end = time.time()
# print('Time Taken:',end-start)
# print('Accuracy is :',accuracy)